In [1]:
import pandas as pd

def load_csv(filepath: str):
    """
    Load a CSV file and perform basic validation.
    - Must include an 'email' column.
    - Trims whitespace.
    - Drops duplicate email rows.
    """
    try:
        df = pd.read_csv(filepath)
    except Exception as e:
        raise ValueError(f"Error reading CSV: {e}")

    # Normalize columns to lowercase for safety
    df.columns = [c.strip().lower() for c in df.columns]

    if "email" not in df.columns:
        raise ValueError("CSV is missing required column: 'email'")

    # Strip whitespace in all string columns
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

    # Drop empty email rows
    df = df[df["email"].notna() & (df["email"] != "")]

    # Drop duplicate emails
    df = df.drop_duplicates(subset=["email"])

    print(f"Loaded CSV with {len(df)} valid rows.")
    return df

# Test helper (you will run this with your file)
def preview_csv(filepath: str):
    df = load_csv(filepath)
    display(df.head())
    return df



In [2]:
preview_csv("sample_emails.csv")

Loaded CSV with 9 valid rows.


,email,first_name,last_name
0,user1@example.com,Alice,Smith
1,user2@example.com,Bob,Jones
2,user3@example.com,Charlie,Brown
3,user4@example.com,David,Wilson
4,user5@example.com,Eve,Taylor


,email,first_name,last_name
0,user1@example.com,Alice,Smith
1,user2@example.com,Bob,Jones
2,user3@example.com,Charlie,Brown
3,user4@example.com,David,Wilson
4,user5@example.com,Eve,Taylor
6,user6@example.com,Grace,Lee
7,user7@example.com,Heidi,Clark
8,user8@example.com,Ivan,Wright
9,user9@example.com,Judy,Walker


In [ ]:
from pydantic import BaseModel, EmailStr, ValidationError
from typing import Optional, List
import pandas as pd

# Define the schema for each recipient
class RecipientRow(BaseModel):
    email: EmailStr               # Required, must be a valid email
    first_name: Optional[str] = None
    last_name: Optional[str] = None
    opt_out: Optional[bool] = False

# Function to validate all rows in a DataFrame
def validate_recipients(df: pd.DataFrame):
    valid_rows = []
    invalid_rows = []

    for idx, row in df.iterrows():
        try:
            # Convert row to dict and validate using Pydantic
            recipient = RecipientRow(**row.to_dict())
            valid_rows.append(recipient)
        except ValidationError as e:
            invalid_rows.append({
                "row_index": idx,
                "errors": e.errors(),
                "data": row.to_dict()
            })

    print(f"✅ Valid rows: {len(valid_rows)}")
    print(f"❌ Invalid rows: {len(invalid_rows)}")
    
    return valid_rows, invalid_rows



In [5]:
df = preview_csv("sample_emails.csv")
valid_recipients, invalid_recipients = validate_recipients(df)


Loaded CSV with 9 valid rows.


,email,first_name,last_name
0,user1@example.com,Alice,Smith
1,user2@example.com,Bob,Jones
2,user3@example.com,Charlie,Brown
3,user4@example.com,David,Wilson
4,user5@example.com,Eve,Taylor


✅ Valid rows: 9
❌ Invalid rows: 0


In [6]:
from jinja2 import Template, meta, Environment

# Function to render template for one recipient
def render_email_template(template_str: str, recipient: dict):
    env = Environment()
    template = env.from_string(template_str)
    
    # Check which placeholders are used
    ast = env.parse(template_str)
    placeholders = meta.find_undeclared_variables(ast)

    missing = [ph for ph in placeholders if ph not in recipient]
    if missing:
        raise ValueError(f"Missing placeholder(s) in recipient data: {missing}")

    return template.render(**recipient)


In [7]:
template = "Hello {{first_name}}, your company {{company}} is amazing!"
recipient = {"first_name": "Alice", "company": "Acme Inc."}

rendered_body = render_email_template(template, recipient)
print(rendered_body)
# Output: "Hello Alice, your company Acme Inc. is amazing!"


Hello Alice, your company Acme Inc. is amazing!
